In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso


warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Prepare the data

In [2]:
# load tabular ukraine data
ukraine_data = pd.read_csv('data/tabular_data_ukraine.csv')

# get training, test, pre_war and prediction data
train_data = ukraine_data[ukraine_data['year'] < 2021]
test_data = ukraine_data[ukraine_data['year'] == 2021]
pre_war_data = ukraine_data[ukraine_data['year'] < 2022]
prediction_data = ukraine_data[ukraine_data['year'] == 2022]

# column_prefixes = ("nearnad_snow_cov", "nearnad_snow_free", "offnad_snow_cov",
#                    "offnad_snow_free", "allangle_snow_cov", "allangle_snow_free", 
#                    "nearnad_snow_free_hq", "offnad_snow_free_hq", "allangle_snow_free_hq")

column_prefixes = ("nearnad_snow_free_hq", "offnad_snow_free_hq", "allangle_snow_free_hq")

# column_prefixes = ("nearnad_snow_free_hq")

general_characteristics = ("mean", "sd")

Define general functions

In [3]:
def build_train_test_sets(selected_columns, train_data, test_data, log_transform = False, scale = False):

    # select columns
    train_data_selected = train_data[["real_gdp", "region", "year"] + selected_columns]
    test_data_selected = test_data[["real_gdp", "region", "year"] + selected_columns]

    if log_transform:
        # real_gdp and columns that contain the word"sum" are log transformed
        train_data_selected["real_gdp"] = np.log(train_data_selected["real_gdp"])
        test_data_selected["real_gdp"] = np.log(test_data_selected["real_gdp"])

        for column in selected_columns:
            if "sum" in column:
                train_data_selected[column] = np.log(train_data_selected[column])
                test_data_selected[column] = np.log(test_data_selected[column])

    if scale:
        # scale the data
        scaler = MinMaxScaler()
        train_data_selected[selected_columns] = scaler.fit_transform(train_data_selected[selected_columns])
        test_data_selected[selected_columns] = scaler.transform(test_data_selected[selected_columns])

        # scale the real_gdp
        train_data_selected["real_gdp"] = scaler.fit_transform(train_data_selected[["real_gdp"]])
        test_data_selected["real_gdp"] = scaler.transform(test_data_selected[["real_gdp"]])

    # one hot encode region
    train_data_selected = pd.get_dummies(train_data_selected, columns=["region"])
    test_data_selected = pd.get_dummies(test_data_selected, columns=["region"])

    return train_data_selected, test_data_selected

def build_model(train_data, test_data, selected_columns, model_type, param_grid, log_transform = False, scale = False):

    # build train and test sets
    train_data_selected, test_data_selected = build_train_test_sets(selected_columns, train_data, test_data, log_transform, scale)

    # get input and output data
    X_train = train_data_selected.drop(columns=["real_gdp", "year"])
    y_train = train_data_selected["real_gdp"]

    X_test = test_data_selected.drop(columns=["real_gdp", "year"])
    y_test = test_data_selected["real_gdp"]

    # build model
    if model_type == "xgboost":
        model_test = xgb.XGBRegressor()
    elif model_type == "random_forest":
        model_test = RandomForestRegressor()
    elif model_type == "lasso":
        model_test = Lasso()

    # hyperparameter tuning
    grid_search = GridSearchCV(estimator=model_test, param_grid=param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    best_params = best_model.get_params()

    # random_search = RandomizedSearchCV(estimator=model_test, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
    # random_search.fit(X_train, y_train)
    # best_model = random_search.best_estimator_

    # make predictions
    y_pred = best_model.predict(X_test)
    
    # calculate mse and mpe
    mae = np.mean(abs(y_pred - y_test))
    mpe = abs(np.mean(100*(y_pred - y_test) / y_test))

    return mae, mpe, best_params


def predict_with_model(pre_war_data, prediction_data, selected_columns, model_type, param_grid, log_transform = False, scale = False):

    # build pre war and prediction sets
    pre_war_data_selected, prediction_data_selected = build_train_test_sets(selected_columns, pre_war_data, prediction_data, log_transform, scale)

    # get input and output data
    X_pre_war = pre_war_data_selected.drop(columns=["real_gdp", "year"])
    y_pre_war = pre_war_data_selected["real_gdp"]
    data_2021 = pre_war_data_selected[pre_war_data_selected["year"] == 2021]

    X_prediction = prediction_data_selected.drop(columns=["real_gdp", "year"])

    # build model, objective: absolute error
    if model_type == "xgboost":
        model_pred = xgb.XGBRegressor()
    elif model_type == "random_forest":
        model_pred = RandomForestRegressor()
    elif model_type == "lasso":
        model_pred = Lasso()

    # hyperparameter tuning
    grid_search = GridSearchCV(estimator=model_pred, param_grid=param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_pre_war, y_pre_war)
    best_model = grid_search.best_estimator_
    best_params = best_model.get_params()

    # make predictions
    y_pred = best_model.predict(X_prediction)

    # calculate the predicted change in the real gdp on the national level
    # if log_transform:
    #     y_pred = np.exp(y_pred)
    pred_gdp_change = 100*(np.sum(y_pred) - np.sum(data_2021["real_gdp"])) / np.sum(data_2021["real_gdp"])

    return pred_gdp_change, best_params

In [4]:
def create_column_names(prefix, general_characteristics):
    general_columns = [prefix + "_" + char for char in general_characteristics]
    log_bin_columns = [prefix + "_log_" + str(i) for i in range(1, 11)] + general_columns
    idr_bin_columns = [prefix + "_idr_" + str(i) for i in range(1, 11)] + general_columns
    
    return general_columns, log_bin_columns, idr_bin_columns

def build_model_and_predict(pre_war_data, prediction_data, selected_columns, model_type, param_grid, log_transform, scale = False, total_metrics = False):

    if total_metrics:
        total_mae = 0
        total_mpe = 0
        
        for year in range(2012, 2022):

            train_data = pre_war_data[pre_war_data['year'] != year]
            test_data = pre_war_data[pre_war_data['year'] == year]     
            mae, mpe, best_params = build_model(train_data, test_data, selected_columns, model_type, param_grid, log_transform, scale)

            total_mae += mae/10
            total_mpe += mpe/10

        gdp_change, best_params = predict_with_model(pre_war_data, prediction_data, selected_columns, model_type, param_grid, log_transform, scale)
        
        return total_mae, total_mpe, gdp_change, best_params

    else:

        train_data = pre_war_data[pre_war_data['year'] != 2021]
        test_data = pre_war_data[pre_war_data['year'] == 2021]
        mae, mpe, _ = build_model(train_data, test_data, selected_columns, model_type, param_grid, log_transform, scale)
        gdp_change, best_params = predict_with_model(pre_war_data, prediction_data, selected_columns, model_type, param_grid, log_transform, scale)

        return mae, mpe, gdp_change, best_params

XGBoost

In [5]:
# Define parameter grid for XGBoost
# param_grid_xgb = {
#     'max_depth': [5, 4, 6],
#     'min_child_weight': [6, 4, 5],
#     'random_state': [0] 
# }

param_grid_xgb = {
    'eta': [0.01, 0.1, 0.2, 0.3, 0.5],
    'gamma': [100, 1000, 10000],
    'max_depth': [4, 6, 8, 10],
    'min_child_weight': [1, 2, 5],
    'random_state': [0] 
}

In [6]:
year = 2021
country_data = pre_war_data
prefix = "allangle_snow_free_hq"
selected_columns =  [prefix + "_" + char for char in general_characteristics] + [prefix + "_log_" + str(i) for i in range(1, 11)]
model_type = "xgboost"
log_transform = False
scale = False

train_data = country_data[country_data['year'] != year]
test_data = country_data[country_data['year'] == year]     
mae, mpe, _ = build_model(train_data, test_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)
gdp_change, best_params = predict_with_model(pre_war_data, prediction_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)


print(gdp_change, best_params)
print(mae, mpe)

KeyboardInterrupt: 

In [7]:
# very promising results with nearnad_snow_free_hq, idr_bin_columns + general_columns (mean, median, sd, sum) with log transform and scale

# initialise a df to store the results
xgb_results = pd.DataFrame(columns=["prefix", "columns", "mae", "mpe", "national_gdp_change"])

for prefix in column_prefixes:
    # create general column names
    general_columns, log_bin_columns, idr_bin_columns = create_column_names(prefix, general_characteristics)
    
    # build xgb models for each year, calculate average mpe and mse, predict the national gdp change, add the results to the df
    for selected_columns, columns_category in zip([log_bin_columns], ["log_bin"]):
        mae, mpe, gdp_change, best_params = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "xgboost", param_grid_xgb, log_transform = False, scale = False, total_metrics = True)
        new_results = pd.DataFrame([{"prefix": prefix, "columns": columns_category, "mae": mae, "mpe": mpe, "national_gdp_change": gdp_change}])
        xgb_results = pd.concat([xgb_results, new_results], ignore_index=True)

    print(f"Finished {prefix}")

# sort by mae, print the results
xgb_results = xgb_results.sort_values(by="mae")
print(xgb_results)
print(best_params)

Finished nearnad_snow_free_hq
Finished offnad_snow_free_hq
Finished allangle_snow_free_hq
                  prefix  columns          mae        mpe  national_gdp_change
1    offnad_snow_free_hq  log_bin  3704.532385   5.695044           -40.886053
2  allangle_snow_free_hq  log_bin  4197.752625   5.017379           -32.501726
0   nearnad_snow_free_hq  log_bin  5857.776513  12.238803           121.917465
{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': 100, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 6, 'max_leaves': None, 'min_child_weight': 5, 'missing': nan, 'monotone_constrain

Random Forest

In [10]:
# Define parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'random_state': [0],  # Ensures reproducibility
    'max_depth': [10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

In [13]:
# initialise a df to store the results
rf_results = pd.DataFrame(columns=["prefix", "columns", "mae", "mpe", "national_gdp_change"])

for prefix in column_prefixes:
    # create general column names
    general_columns, log_bin_columns, idr_bin_columns = create_column_names(prefix, general_characteristics)
    
    # build xgb models and predict the national gdp change, add the results to the df
    for selected_columns, columns_category in zip([log_bin_columns], ["log_bin"]):
        mae, mpe, gdp_change, best_params = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False, total_metrics = True)
        new_results = pd.DataFrame([{"prefix": prefix, "columns": columns_category, "mae": mae, "mpe": mpe, "national_gdp_change": gdp_change}])
        rf_results = pd.concat([rf_results, new_results], ignore_index=True)

    print(f"Finished {prefix}")

# sort by mae, print the results
rf_results = rf_results.sort_values(by="mae")
print(rf_results)
print(best_params)

Finished nearnad_snow_free_hq
Finished offnad_snow_free_hq
Finished allangle_snow_free_hq
                  prefix  columns          mae       mpe  national_gdp_change
2  allangle_snow_free_hq  log_bin  3694.764568  3.246507           -31.886836
1    offnad_snow_free_hq  log_bin  3807.619965  2.319833           -42.694637
0   nearnad_snow_free_hq  log_bin  3809.765291  3.103696            -2.157626
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 20, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Lasso

In [37]:
# Define parameter grid for Lasso
param_grid_lasso = {
    'alpha': [0.1, 0.5, 1]    
}

In [38]:
# initialise a df to store the results
lasso_results = pd.DataFrame(columns=["prefix", "columns", "mae", "mpe", "national_gdp_change"])

for prefix in column_prefixes:
    # create general column names
    general_columns, log_bin_columns, idr_bin_columns = create_column_names(prefix, general_characteristics)
    
    # build xgb models and predict the national gdp change, add the results to the df
    for selected_columns, columns_category in zip([general_columns, log_bin_columns, idr_bin_columns], ["general", "log_bin", "idr_bin"]):
        mae, mpe, gdp_change = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "lasso", param_grid_lasso, log_transform = True, scale = True, total_metrics = True)
        new_results = pd.DataFrame([{"prefix": prefix, "columns": columns_category, "mae": mae, "mpe": mpe, "national_gdp_change": gdp_change}])
        lasso_results = pd.concat([lasso_results, new_results], ignore_index=True)

    print(f"Finished {prefix}")

# sort by mae, print the results
lasso_results = lasso_results.sort_values(by="mae")
print(lasso_results)

Finished nearnad_snow_free_hq
Finished offnad_snow_free_hq
Finished allangle_snow_free_hq
                  prefix  columns       mae       mpe  national_gdp_change
7  allangle_snow_free_hq  log_bin  1.995504  2.485594           -49.219083
1   nearnad_snow_free_hq  log_bin  2.018039  3.405903           -45.145320
8  allangle_snow_free_hq  idr_bin  2.033258  3.242828           -50.658702
6  allangle_snow_free_hq  general  2.046027  2.207463           -57.345137
2   nearnad_snow_free_hq  idr_bin  2.068528  4.930689           -45.672186
4    offnad_snow_free_hq  log_bin  2.087120  2.039300           -44.180048
0   nearnad_snow_free_hq  general  2.146417  3.630255           -47.425399
5    offnad_snow_free_hq  idr_bin  2.160240  2.726171           -51.422675
3    offnad_snow_free_hq  general  2.227228  4.855434           -42.026040
